In [1]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go

In [2]:
from glob import glob

In [13]:
import plotly
import plotly.offline as py
import plotly.graph_objs as go

In [14]:
files = glob(f'data/STK/1_min/TSLA/20210501.csv')
data_list = []
for file in files:
    data_list.append(pd.read_csv(file))
df = pd.concat(data_list).reset_index(drop=True)

* check the trend change on before 12

In [15]:
df['date'] = pd.to_datetime(df['date'])
df.index = df['date']
df = df.between_time('21:30', '23:59')
df['time'] = df['date'].dt.time
df['date'] = df['date'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [16]:
valid_dates = df['date'].value_counts()

In [17]:
valid_dates = list(valid_dates[valid_dates > 3].index)

In [18]:
df = df[df['date'].isin(valid_dates)]

In [19]:
df.head()

,date,open,high,low,close,volume,barCount,average,time
date,,,,,,,,,
2021-04-30 21:30:00,2021-04-30,667.59,670.00,666.14,668.11,2390,573,668.009,21:30:00
2021-04-30 21:31:00,2021-04-30,667.72,672.73,667.52,671.23,1179,587,670.746,21:31:00
2021-04-30 21:32:00,2021-04-30,671.60,672.00,669.29,670.69,695,370,670.606,21:32:00
2021-04-30 21:33:00,2021-04-30,670.82,671.52,669.86,670.39,550,303,670.675,21:33:00
2021-04-30 21:34:00,2021-04-30,670.94,671.60,669.50,670.00,532,333,670.607,21:34:00


# Candle Chart

In [20]:
dates = sorted(df['date'].unique())

In [21]:
fig = go.Figure()

for date in dates:
    data = df[df['date'] == date]
    fig.add_trace(go.Candlestick(x=data['time'],
            open=data['open'],
            high=data['high'],
            low=data['low'],
            close=data['close'],
                                name = date))

import plotly.io as pio
pio.renderers.default = 'browser'
pio.show(fig)

# Line Chart

In [13]:
line_data = df[['date', 'time' ,'average', 'open']].copy()

In [14]:
# Find the min open price
min_open_price = line_data['open'].min()

In [15]:
# To bring all open price to 0, subtract open price from average
line_data['avg_nor'] = line_data['average'] - line_data['open']

In [16]:
dates = sorted(line_data['date'].unique())

In [17]:
fig = go.Figure()

for date in dates:
    data = line_data[line_data['date'] == date]
    fig.add_trace(go.Scatter(x=data['time'], y=data['avg_nor'], name = date))

import plotly.io as pio
pio.renderers.default = 'browser'
pio.show(fig)